In [26]:
from huggingface_hub import login
from sam3.model_builder import build_sam3_image_model
from sam3.model.sam3_image_processor import Sam3Processor
import numpy as np
import matplotlib

In [27]:
def overlay_masks(image, masks):
    image = image.convert("RGBA")
    masks = 255 * masks.cpu().numpy().astype(np.uint8)
    
    n_masks = masks.shape[0]
    cmap = matplotlib.colormaps.get_cmap("rainbow").resampled(n_masks)
    colors = [
        tuple(int(c * 255) for c in cmap(i)[:3])
        for i in range(n_masks)
    ]

    for mask, color in zip(masks, colors):
        mask = Image.fromarray(mask)
        overlay = Image.new("RGBA", image.size, color + (0,))
        alpha = mask.point(lambda v: int(v * 0.5))
        overlay.putalpha(alpha)
        image = Image.alpha_composite(image, overlay)
    return image

In [3]:
login()
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
model = build_sam3_image_model()

sam3.pt:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

In [10]:
processor = Sam3Processor(model)

In [21]:
# Load an image
image = Image.open("../data/truck.jpg")
inference_state = processor.set_image(image)

In [22]:
# Prompt the model with text
output = processor.set_text_prompt(state=inference_state, prompt="truck")

In [23]:
# Get the masks, bounding boxes, and scores
masks, boxes, scores = output["masks"], output["boxes"], output["scores"]

In [24]:
masks, boxes, scores

(tensor([[[[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]]]], device='cuda:0'),
 tensor([[  85.4299,  281.5704, 1710.3721,  850.5768]], device='cuda:0'),
 tensor([0.8647], device='cuda:0'))

In [28]:
overlay_masks(image, masks)

TypeError: Cannot handle this data type: (1, 1, 1800), |u1